In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os
print(os.getcwd())

c:\Users\rohan\Desktop\book-recommendation-project\notebooks


In [5]:
books = pd.read_csv('../data/raw/Books.csv')
users = pd.read_csv('../data/raw/Users.csv')
ratings = pd.read_csv('../data/raw/Ratings.csv')


C:\Users\rohan\AppData\Local\Temp\ipykernel_14432\370617393.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('../data/raw/Books.csv')


In [6]:
# Load the three CSV files
books = pd.read_csv('../data/raw/Books.csv')
users = pd.read_csv('../data/raw/Users.csv')
ratings = pd.read_csv('../data/raw/Ratings.csv')

C:\Users\rohan\AppData\Local\Temp\ipykernel_14432\3235032442.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('../data/raw/Books.csv')


In [7]:
# Basic info about each dataset
print("Books shape:", books.shape)
print("Users shape:", users.shape) 
print("Ratings shape:", ratings.shape)

Books shape: (271360, 8)
Users shape: (278858, 3)
Ratings shape: (1149780, 3)


In [8]:
# Display first few rows
print("\n--- BOOKS ---")
print(books.head())
print("\n--- USERS ---")
print(users.head())
print("\n--- RATINGS ---")
print(ratings.head())


--- BOOKS ---
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://ima

In [9]:
# Check for missing values
print("Books missing values:")
print(books.isnull().sum())

print("\nUsers missing values:")
print(users.isnull().sum())

print("\nRatings missing values:")
print(ratings.isnull().sum())

# Check data types
print("\nBooks data types:")
print(books.dtypes)


Books missing values:
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

Users missing values:
User-ID          0
Location         0
Age         110762
dtype: int64

Ratings missing values:
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Books data types:
ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object


In [10]:
# Remove books with missing essential information
books_clean = books.dropna(subset=['Book-Title', 'Book-Author'])

# Handle year of publication issues
books_clean['Year-Of-Publication'] = pd.to_numeric(books_clean['Year-Of-Publication'], errors='coerce')
books_clean = books_clean[(books_clean['Year-Of-Publication'] >= 1900) & 
                         (books_clean['Year-Of-Publication'] <= 2024)]

# Clean ISBN (remove duplicates)
books_clean = books_clean.drop_duplicates(subset=['ISBN'])


C:\Users\rohan\AppData\Local\Temp\ipykernel_14432\2714536943.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_clean['Year-Of-Publication'] = pd.to_numeric(books_clean['Year-Of-Publication'], errors='coerce')


In [11]:
# Filter out implicit ratings (0) for now, focus on explicit ratings (1-10)
ratings_explicit = ratings[ratings['Book-Rating'] > 0]

# Remove ratings for books that don't exist in our cleaned books dataset
valid_isbns = set(books_clean['ISBN'])
ratings_clean = ratings_explicit[ratings_explicit['ISBN'].isin(valid_isbns)]

print(f"Original ratings: {len(ratings)}")
print(f"Explicit ratings: {len(ratings_explicit)}")
print(f"Clean ratings: {len(ratings_clean)}")


Original ratings: 1149780
Explicit ratings: 433671
Clean ratings: 378029


In [12]:
# Start with users who have rated at least 20 books
user_counts = ratings_clean['User-ID'].value_counts()
active_users = user_counts[user_counts >= 20].index

# Start with books that have at least 50 ratings
book_counts = ratings_clean['ISBN'].value_counts()
popular_books = book_counts[book_counts >= 50].index

# Create filtered dataset
ratings_filtered = ratings_clean[
    (ratings_clean['User-ID'].isin(active_users)) &
    (ratings_clean['ISBN'].isin(popular_books))
]

print(f"Filtered dataset size: {len(ratings_filtered)}")
print(f"Unique users: {ratings_filtered['User-ID'].nunique()}")
print(f"Unique books: {ratings_filtered['ISBN'].nunique()}")


Filtered dataset size: 22383
Unique users: 2800
Unique books: 529


In [13]:
from scipy.sparse import csr_matrix

# Create pivot table (user-item matrix)
user_item_matrix = ratings_filtered.pivot(
    index='User-ID', 
    columns='ISBN', 
    values='Book-Rating'
).fillna(0)

print("User-item matrix shape:", user_item_matrix.shape)

# Convert to sparse matrix for memory efficiency
user_item_sparse = csr_matrix(user_item_matrix.values)


User-item matrix shape: (2800, 529)


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate item-item similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

def get_similar_books(isbn, top_n=5):
    """Get top N similar books for a given ISBN"""
    if isbn not in item_similarity_df.index:
        return "Book not found"
    
    similar_scores = item_similarity_df[isbn].sort_values(ascending=False)
    similar_books = similar_scores[1:top_n+1]  # Exclude the book itself
    
    # Get book titles
    book_titles = []
    for book_isbn in similar_books.index:
        title = books_clean[books_clean['ISBN'] == book_isbn]['Book-Title'].iloc[0]
        book_titles.append((book_isbn, title, similar_scores[book_isbn]))
    
    return book_titles

# Test the function
sample_isbn = ratings_filtered['ISBN'].iloc[0]
similar_books = get_similar_books(sample_isbn)
print("Similar books:", similar_books)


Similar books: [('0316666009', '1st to Die: A Novel', np.float64(0.12776347624027865)), ('1844262553', 'Free', np.float64(0.1216910871472255)), ('080411109X', 'The Hundred Secret Senses', np.float64(0.1104880440386104)), ('0679745203', 'The English Patient', np.float64(0.10085645518789128)), ('0316096199', 'Lucky : A Memoir', np.float64(0.10007686644145722))]


In [15]:
def recommend_books_for_user(user_id, top_n=5):
    """Recommend books for a specific user"""
    if user_id not in user_item_matrix.index:
        return "User not found"
    
    # Get books rated by the user
    user_ratings = user_item_matrix.loc[user_id]
    rated_books = user_ratings[user_ratings > 0].index
    
    # Calculate recommendations based on similar books
    recommendations = {}
    
    for book_isbn in rated_books:
        user_rating = user_ratings[book_isbn]
        similar_books = get_similar_books(book_isbn, 10)
        
        for sim_isbn, title, similarity in similar_books:
            if sim_isbn not in rated_books:  # Don't recommend already rated books
                score = user_rating * similarity
                if sim_isbn in recommendations:
                    recommendations[sim_isbn] += score
                else:
                    recommendations[sim_isbn] = score
    
    # Sort and get top N
    top_recommendations = sorted(recommendations.items(), 
                               key=lambda x: x[1], 
                               reverse=True)[:top_n]
    
    # Get book titles
    final_recommendations = []
    for isbn, score in top_recommendations:
        title = books_clean[books_clean['ISBN'] == isbn]['Book-Title'].iloc[0]
        author = books_clean[books_clean['ISBN'] == isbn]['Book-Author'].iloc[0]
        final_recommendations.append((isbn, title, author, score))
    
    return final_recommendations

# Test recommendations
sample_user = ratings_filtered['User-ID'].iloc[0]
user_recommendations = recommend_books_for_user(sample_user)
print("Recommendations for user:", user_recommendations)


Recommendations for user: [('1844262553', 'Free', 'Paul Vincent', np.float64(6.541812050461187)), ('0553280341', 'B Is for Burglar (Kinsey Millhone Mysteries (Paperback))', 'Sue Grafton', np.float64(3.958675600877975)), ('0671001795', 'Two for the Dough', 'Janet Evanovich', np.float64(3.9380563261749284)), ('0449219461', 'H Is for Homicide (Kinsey Millhone Mysteries (Paperback))', 'Sue Grafton', np.float64(3.3935612615935176)), ('1400031354', 'Tears of the Giraffe (No.1 Ladies Detective Agency)', 'Alexander McCall Smith', np.float64(3.2184590748727517))]


In [17]:
import os
import pickle

# Create the models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save processed data and similarity matrix
with open('models/item_similarity.pkl', 'wb') as f:
    pickle.dump(item_similarity_df, f)

with open('models/user_item_matrix.pkl', 'wb') as f:
    pickle.dump(user_item_matrix, f)


In [18]:
import pickle

# Save processed data and similarity matrix
with open('models/item_similarity.pkl', 'wb') as f:
    pickle.dump(item_similarity_df, f)

with open('models/user_item_matrix.pkl', 'wb') as f:
    pickle.dump(user_item_matrix, f)

with open('models/books_clean.pkl', 'wb') as f:
    pickle.dump(books_clean, f)

print("Models saved successfully!")


Models saved successfully!
